In [7]:
import sys
sys.path.append("..")
import random
from sklearn.model_selection import train_test_split
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation

from movingpose.estimator import neighbors
from movingpose.estimator import classifiers

from movingpose.preprocessing import moving_pose
from movingpose.preprocessing import kinect_skeleton_data

## Pickle multiview action data

In [8]:
kinect_skeleton_data.pickle_dir("../pickle/multiview.p", "../ext/dataset/multiview_action/")

Pickling skeleton data...
Saving data...
Saved data to file: ../pickle/multiview.p
Program took 14.822474002838135 to run!


## Load pickled multiview action data

In [9]:
raw_data_dict = kinect_skeleton_data.load_pickle("../pickle/multiview.p")

## Format multiview action data

In [10]:
X, labels = moving_pose.format_skeleton_data_dict(raw_data_dict)

## Visualize formatted skeleton data

In [27]:
# initialize joint colors of skeleton
colors = ['blue' for i in range(20)]
colors[0] = 'red'  # hip is red
colors[7] = 'green'  # hands and feet are green
colors[11] = 'green'
colors[15] = 'green'
colors[19] = 'green'
colors[3] = 'black' # head is black

action_data = X[random.randrange(0, len(X))]

for i in range(0, len(action_data), 20):
    assert action_data[i][:-1] == [0, 0, 0, 0, 0, 0, 0, 0, 0]
    print(action_data[i][-1])

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

## Create train/test split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, random_state=42)

sup


## Train Action Classifier with normalized training data

In [7]:
best_score = sys.maxsize
best_action_classifier = None
for n_neighbors in range(5, 15, 5):
    for n_training_neighbors in range(10, 40, 10):
        for alpha in np.arange(0.2, 0.8, 0.1):
            for beta in np.arange(0.1, 0.5, 0.1):
                for kappa in range(3, 10, 1):
                    nearest_descriptor_estimator = neighbors.NearestDescriptors(
                        n_neighbors=n_neighbors,
                        n_training_neighbors=n_training_neighbors,
                        alpha=alpha,
                        beta=beta,
                        kappa=kappa
                    )
                    for theta in np.arange(0.2, 0.8, 0.05):
                        for n in range(5, 15, 1):
                            action_classifier = classifiers.ActionClassifier(
                                nearest_descriptor_estimator=nearest_descriptor_estimator,
                                theta=theta,
                                n=n
                            )
                            action_classifier.fit(X_train, y_train)
                            y_pred = action_classifier.predict(X_test)

                            diff = 0
                            for pred, actual in zip(y_pred, y_test):
                                diff += abs(pred - actual)

                            if diff < best_score:
                                best_score = diff
                                best_action_classifier = action_classifier

KeyboardInterrupt: 

## Print best action classifier and its score

In [ ]:
print(f"The best action classifier was {best_action_classifier}")
print("\n--------------\n")
print(f"Its score was: {best_score}")

## Save the best action classifier

In [ ]:
best_action_classifier.save_pickle("../pickle/best_action_classifier.p")

## Load best action classifier

In [ ]:
action_classifier = classifiers.load_pickle("../pickle/best_action_classifier.p")

## Predict live data

In [ ]:
#TODO add live predictions when mehmet publishes the UI code

# DAB

In [ ]:
print("dab")
